# Compare prepbufr_decoder and IODA Output
  
shawn.s.murdzek@noaa.gov  
  
Machine: Jet  
Environment: adb_graphics

In [1]:
import numpy as np
import xarray as xr
import pyDA_utils.bufr as bufr

In [53]:
"""
Read in Datasets
"""

path = '/lfs4/BMC/wrfruc/murdzek/real_obs/decoded_bufr_for_cloudDA/'

original_ioda = {}
ioda_groups = ['MetaData', 'ObsError', 'ObsType', 'ObsValue', 'QualityMarker', 'TunedObsError']
for group in ioda_groups:
    original_ioda[group] = xr.open_dataset(path + '2022020109.rap.t09z.prepbufr.surface.nc', group=group)
original_decoder_csv = bufr.bufrCSV(path + '202202010900.rap.prepbufr.csv').df

/contrib/miniconda3/4.5.12/envs/adb_graphics/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [54]:
"""
Only Focus on prepBUFR Types That Exist in the IODA File
"""

typs = np.unique(original_ioda['MetaData']['prepbufrReportType'])
print('prepbufr types =', typs)
keep_idx = np.zeros(len(original_decoder_csv))
for t in typs:
    keep_idx[original_decoder_csv['TYP'] == t] = 1
intermediate_csv = original_decoder_csv.loc[keep_idx == 1, :].copy()
intermediate_csv.reset_index(inplace=True)

prepbufr types = [180. 181. 183. 187. 192. 193. 194. 280. 281. 282. 284. 287. 292. 293.
 294.]


In [55]:
"""
Only Focus on a Certain Geographical Region
"""

# San Antonio region
lat_bounds = [29.3, 29.7]
lon_bounds = [-98.6, -98.3]

ioda_idx = np.where((original_ioda['MetaData']['latitude'] >= lat_bounds[0]) &
                    (original_ioda['MetaData']['latitude'] <= lat_bounds[1]) &
                    (original_ioda['MetaData']['longitude'] >= (360 + lon_bounds[0])) &
                    (original_ioda['MetaData']['longitude'] <= (360 + lon_bounds[1])))[0]

ioda_data = {}
for group in ioda_groups:
    ioda_data[group] = original_ioda[group].sel(Location=ioda_idx)


decoder_idx = np.where((intermediate_csv['YOB'] >= lat_bounds[0]) &
                       (intermediate_csv['YOB'] <= lat_bounds[1]) &
                       (intermediate_csv['XOB'] >= (360 + lon_bounds[0])) &
                       (intermediate_csv['XOB'] <= (360 + lon_bounds[1])))[0]

decoder_csv = intermediate_csv.iloc[decoder_idx, :].copy()

In [60]:
ioda_data['MetaData']

<xarray.Dataset>
Dimensions:                (Location: 12)
Coordinates:
    latitude               (Location) float32 29.33 29.33 29.33 ... 29.53 29.53
    longitude              (Location) float32 261.53 261.53 ... 261.53 261.53
Dimensions without coordinates: Location
Data variables:
    dateTime               (Location) datetime64[ns] 2022-02-01T08:53:00 ... 2022-02-01T08:51:00
    dumpReportType         (Location) float64 512.0 512.0 512.0 ... 512.0 512.0
    height                 (Location) float64 nan 176.0 nan ... 210.0 nan 242.0
    prepbufrDataLvlCat     (Location) float64 6.0 0.0 6.0 0.0 ... 0.0 6.0 0.0
    prepbufrReportType     (Location) float64 287.0 187.0 287.0 ... 287.0 187.0
    pressure               (Location) float32 99030.0 99030.0 ... 98280.0
    seaTemperatureMethod   (Location) float64 nan nan nan nan ... nan nan nan
    stationElevation       (Location) float64 176.0 176.0 176.0 ... 242.0 242.0
    stationIdentification  (Location) object KSSF KSSF KSSF ... KSKF KSAT KSAT

In [61]:
ioda_data['ObsValue']

<xarray.Dataset>
Dimensions:                                (Location: 12)
Dimensions without coordinates: Location
Data variables:
    airTemperature                         (Location) float32 ...
    cloudAmount                            (Location) float64 ...
    cloudCeiling                           (Location) float32 ...
    cloudCoverTotal                        (Location) float32 ...
    depthBelowSeaSurface                   (Location) float32 ...
    dewpointTemperature                    (Location) float32 ...
    heightAboveSurfaceOfBaseOfLowestCloud  (Location) float64 ...
    heightOfBaseOfCloud                    (Location) float32 ...
    heightOfWaves                          (Location) float32 ...
    horizontalVisibility                   (Location) float32 ...
    maximumTemperature                     (Location) float32 ...
    maximumWindGustSpeed                   (Location) float32 ...
    minimumTemperature                     (Location) float32 ...
    presentWeather                         (Location) float64 ...
    pressureReducedToMeanSeaLevel          (Location) float32 ...
    seaTemperature                         (Location) float32 ...
    specificHumidity                       (Location) float32 ...
    verticalSignificance                   (Location) float64 ...
    windEastward                           (Location) float32 ...
    windNorthward                          (Location) float32 ...

In [62]:
decoder_csv.loc[:, ['SID', 'XOB', 'YOB', 'DHR', 'TYP', 'POB', 'ELV', 'TOB', 'QOB', 'UOB', 'VOB', 
                    'CLAM', 'HOCB', 'CEILING','PRWE']]

,SID,XOB,YOB,DHR,TYP,POB,ELV,TOB,QOB,UOB,VOB,CLAM,HOCB,CEILING,PRWE
1918,KSSF,261.53,29.33,-0.11667,287.0,990.3,176.0,NaN,NaN,-1.8,-1.0,NaN,NaN,NaN,NaN
1919,KSSF,261.53,29.33,-0.11667,187.0,990.3,176.0,15.6,9649.0,NaN,NaN,8.0,460.0,460.0,110.0
1920,KSSF,261.53,29.33,0.01667,287.0,989.9,176.0,NaN,NaN,-1.8,-1.0,NaN,NaN,NaN,NaN
1921,KSSF,261.53,29.33,0.01667,187.0,989.9,176.0,15.6,9653.0,NaN,NaN,12.0,400.0,400.0,NaN
1922,KSSF,261.53,29.33,0.01667,187.0,NaN,176.0,NaN,NaN,NaN,NaN,8.0,580.0,NaN,NaN
1923,KSSF,261.53,29.33,0.25000,287.0,989.9,176.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1924,KSSF,261.53,29.33,0.25000,187.0,989.9,176.0,15.6,9653.0,NaN,NaN,11.0,400.0,640.0,NaN
1925,KSSF,261.53,29.33,0.25000,187.0,NaN,176.0,NaN,NaN,NaN,NaN,8.0,640.0,NaN,NaN
1947,KSKF,261.42,29.38,-0.23333,287.0,985.9,210.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1948,KSKF,261.42,29.38,-0.23333,187.0,985.9,210.0,13.0,9503.0,NaN,NaN,11.0,180.0,430.0,NaN


In [63]:
decoder_csv['TOB'] + 273.15

1918       NaN
1919    288.75
1920       NaN
1921    288.75
1922       NaN
1923       NaN
1924    288.75
1925       NaN
1947       NaN
1948    286.15
1949       NaN
1950       NaN
1951    286.65
1952       NaN
1975       NaN
1976    287.05
1977       NaN
Name: TOB, dtype: float64